## Imports

In [1]:
import pandas as pd
from recipe_scrapers import scrape_me
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
import dill as pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
from sklearn.neighbors import NearestNeighbors

## Functions

In [2]:
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [3]:
def ingredient_parcer(recipe):
    
    #recipe = ' '.join(recipe)
    
    recipe = recipe.lower()
    
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenizer = RegexpTokenizer(pattern)
    recipe = tokenizer.tokenize(recipe)
    
    recipe_tagged = pos_tag(recipe)
    recipe_tagged
    
    new_list = []
    for i in recipe_tagged:
        if i[1]== 'NN':
            new_list.append(i)
        elif i[1]== 'NNS':
            new_list.append(i)
#         elif i[1] == 'JJ':
#             new_list.append(i)
    
    n_list = []
    for i in new_list:
        n_list.append(i[0])
     
    recipe = [token for token in n_list if token not in sw]
    recipe = ' '.join(recipe)
    return recipe

### Load data

In [6]:
with open('ingredients.pickle', 'rb') as recipes:
       ingredients = pickle.load(recipes)

In [7]:
with open('recipe_names.pickle', 'rb') as URL:
       recipe_url = pickle.load(URL)

In [8]:
# ingredients = []
# recipe_names = []
# for i in list(range(101, 1023174)):
#     scraper = scrape_me(f'https://cooking.nytimes.com/recipes/{i}')
#     ingredients.append(scraper.ingredients())
#     recipe_names.append(scraper.canonical_url())

In [9]:
ingredients[0:5]

[['1 1/2 pounds new potatoes (2 cups squeezed)',
  '8 ounces onion ( 1/2 cup squeezed)',
  '3 egg whites',
  '2 tablespoons flour',
  '5 tablespoons grated Parmigiano Reggiano cheese (2 ounces)',
  '1/8 teaspoon salt',
  'Freshly ground black pepper to taste',
  'Oil spray'],
 ['1 1/2 pounds zucchini',
  '8 ounces onion ( 1/2 cup squeezed)',
  '3 tablespoons flour',
  '4 egg whites',
  '1/2 teaspoon dried thyme leaves',
  '1/4 teaspoon salt',
  'Freshly ground black pepper to taste'],
 ['Butter for greasing a pan',
  'Salt to taste if desired',
  'Freshly ground pepper to taste',
  '4 flounder fillets, about 1 1/4 pounds',
  '1 tablespoon plus 1/3 cup dry white wine',
  '2 cups parsley leaves, stems removed, loosely packed',
  '2 tablespoons finely chopped shallots',
  '1/3 cup heavy cream',
  '1 teaspoon Dijon-style mustard'],
 ['2 large cucumbers, about 1 1/4 pounds',
  '2 carrots, about 1/2 pound',
  '2 tablespoons butter',
  '1 tablespoon finely chopped shallots',
  'Salt to taste 

In [10]:
len(ingredients)

9021

### Putting data together

In [11]:
recipe_url[0][41:].replace('-',' ').title() # needs to be parsed out

'Potato Cheese Pancakes'

In [12]:
recipe_name = []
for i in recipe_url:
    recipe_name.append(i[41:])
recipe_name[0:5]    

['potato-cheese-pancakes',
 'zucchini-pancakes',
 'fillets-of-flounder-with-parsley-and-mustard-sauce',
 'cucumbers-and-carrots',
 'pasta-with-fennel-and-sardines']

In [13]:
Titles = []
for i in recipe_name:
    Titles.append(i.replace('-',' ').title())
    
Titles[0:5]   

['Potato Cheese Pancakes',
 'Zucchini Pancakes',
 'Fillets Of Flounder With Parsley And Mustard Sauce',
 'Cucumbers And Carrots',
 'Pasta With Fennel And Sardines']

In [14]:
urls = pd.DataFrame(recipe_url)
urls.columns = ['URL']

In [15]:
titles = pd.DataFrame(Titles)
titles.columns= ['Titles']

Looking at the ingredient list

In [16]:
sample_recipe = ingredients[0]
sample_recipe

['1 1/2 pounds new potatoes (2 cups squeezed)',
 '8 ounces onion ( 1/2 cup squeezed)',
 '3 egg whites',
 '2 tablespoons flour',
 '5 tablespoons grated Parmigiano Reggiano cheese (2 ounces)',
 '1/8 teaspoon salt',
 'Freshly ground black pepper to taste',
 'Oil spray']

The ingredients are a list of strings. I think parcing out the important information will be easier if I join the strings into one string to avoid an imbedded for loop.


In [17]:
sample_recipe = ' '.join(ingredients[0])
sample_recipe

'1 1/2 pounds new potatoes (2 cups squeezed) 8 ounces onion ( 1/2 cup squeezed) 3 egg whites 2 tablespoons flour 5 tablespoons grated Parmigiano Reggiano cheese (2 ounces) 1/8 teaspoon salt Freshly ground black pepper to taste Oil spray'

In [18]:
test_list = []
for sublist in ingredients:
    sample = ' '.join(sublist)
    test_list.append(sample)

In [19]:
ingredients_list = pd.DataFrame(test_list)
ingredients_list.columns = ['Ingredients']
ingredients_list.head()

,Ingredients
0,1 1/2 pounds new potatoes (2 cups squeezed) 8 ...
1,1 1/2 pounds zucchini 8 ounces onion ( 1/2 cup...
2,Butter for greasing a pan Salt to taste if des...
3,"2 large cucumbers, about 1 1/4 pounds 2 carrot..."
4,2 packed cups mixed greens of fresh fennel fro...


In [20]:
df = urls.join(ingredients_list,lsuffix='URL',rsuffix='Ingredients')

In [21]:
df = titles.join(df,lsuffix='Titles',rsuffix="URL")

In [22]:
df.head()

,Titles,URL,Ingredients
0,Potato Cheese Pancakes,https://cooking.nytimes.com/recipes/1000-potat...,1 1/2 pounds new potatoes (2 cups squeezed) 8 ...
1,Zucchini Pancakes,https://cooking.nytimes.com/recipes/1001-zucch...,1 1/2 pounds zucchini 8 ounces onion ( 1/2 cup...
2,Fillets Of Flounder With Parsley And Mustard S...,https://cooking.nytimes.com/recipes/1002-fille...,Butter for greasing a pan Salt to taste if des...
3,Cucumbers And Carrots,https://cooking.nytimes.com/recipes/1003-cucum...,"2 large cucumbers, about 1 1/4 pounds 2 carrot..."
4,Pasta With Fennel And Sardines,https://cooking.nytimes.com/recipes/1004-pasta...,2 packed cups mixed greens of fresh fennel fro...


In [23]:
df[df['Titles']== '']

,Titles,URL,Ingredients
192,,https://cooking.nytimes.com/recipes/1192,
315,,https://cooking.nytimes.com/recipes/1315,
372,,https://cooking.nytimes.com/recipes/1372,
425,,https://cooking.nytimes.com/recipes/1425,
441,,https://cooking.nytimes.com/recipes/1441,
...,...,...,...
8326,,https://cooking.nytimes.com/recipes/9326,
8386,,https://cooking.nytimes.com/recipes/9386,
8414,,https://cooking.nytimes.com/recipes/9414,
8415,,https://cooking.nytimes.com/recipes/9415,


This is where I would remove the rows with empty strings but that affected my parsing function so I am commenting it out.

In [24]:
#df.replace('', np.nan,inplace=True)

In [25]:
#df[df['Titles'].isna()]

In [26]:
#df.isnull().sum()

In [27]:
#f.dropna(subset=["Titles"],inplace=True)

In [28]:
#df.isnull().sum()

I want to now start using NLP methods to reduce the words in the string to a simplified string of ingredients.


In [29]:
sample_recipe = sample_recipe.lower() # turn all letter to lower case
sample_recipe

'1 1/2 pounds new potatoes (2 cups squeezed) 8 ounces onion ( 1/2 cup squeezed) 3 egg whites 2 tablespoons flour 5 tablespoons grated parmigiano reggiano cheese (2 ounces) 1/8 teaspoon salt freshly ground black pepper to taste oil spray'

In [30]:
# Using Regex to tokenize the sting 
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
sample_recipe = tokenizer.tokenize(sample_recipe)
sample_recipe

['pounds',
 'new',
 'potatoes',
 'cups',
 'squeezed',
 'ounces',
 'onion',
 'cup',
 'squeezed',
 'egg',
 'whites',
 'tablespoons',
 'flour',
 'tablespoons',
 'grated',
 'parmigiano',
 'reggiano',
 'cheese',
 'ounces',
 'teaspoon',
 'salt',
 'freshly',
 'ground',
 'black',
 'pepper',
 'to',
 'taste',
 'oil',
 'spray']

In [31]:
# tagging POS to see if unnecessary parts of speech can be taken out
sample_recipe_tagged = pos_tag(sample_recipe)
sample_recipe_tagged

[('pounds', 'NNS'),
 ('new', 'JJ'),
 ('potatoes', 'NNS'),
 ('cups', 'NNS'),
 ('squeezed', 'VBD'),
 ('ounces', 'NNS'),
 ('onion', 'JJ'),
 ('cup', 'NN'),
 ('squeezed', 'VBD'),
 ('egg', 'NN'),
 ('whites', 'NNS'),
 ('tablespoons', 'NNS'),
 ('flour', 'VBP'),
 ('tablespoons', 'NNS'),
 ('grated', 'VBD'),
 ('parmigiano', 'JJ'),
 ('reggiano', 'NN'),
 ('cheese', 'JJ'),
 ('ounces', 'NNS'),
 ('teaspoon', 'VBP'),
 ('salt', 'JJ'),
 ('freshly', 'RB'),
 ('ground', 'NN'),
 ('black', 'JJ'),
 ('pepper', 'NN'),
 ('to', 'TO'),
 ('taste', 'VB'),
 ('oil', 'NN'),
 ('spray', 'NN')]

Looks like there won't be a simple solution from tagging because labeling POS isn't perfect. If I just wanted to keep nouns, which would make logical sense for a list of ingredients, some of the nouns are labeled as adjectives.
In this case onion, flour, salt and cheese are all not tagged as nouns.

For a more targeted way to remove problematic words, building up a good stopword list can eliminate predicatable words. I found a similar project on GitHub and took/edited some of the stopwords lists they had built. Found 
[here](https://github.com/jackmleitch/whatscooking-deployment/blob/master/ingredient_parser.py). 

In [32]:
measures = [
        "teaspoon",
        "teaspoons",
        "t",
        "tsp.",
        "tablespoon",
        "tablespoons",
        "T",
        "tbl.",
        "tb",
        "tbsp.",
        "fluid",
        "ounce",
        "ounces",
        "fl oz",
        "gill",
        "cup",
        "cups",
        "c",
        "pint",
        "pints",
        "p",
        "pt",
        "fl pt",
        "quart",
        "quarts",
        "q",
        "qt",
        "fl qt",
        "gallon",
        "gallons",
        "g",
        "gal",
        "ml",
        "milliliter",
        "milliliters",
        "millilitre",
        "cc",
        "mL",
        "l",
        "liter",
        "liters",
        "litre",
        "L",
        "dl",
        "deciliter",
        "decilitre",
        "dL",
        "bulb",
        "level",
        "heaped",
        "rounded",
        "whole",
        "pinch",
        "medium",
        "slice",
        "pound",
        "pounds",
        "lb",
        "#",
        "ounce",
        "oz",
        "mg",
        "milligram",
        "milligrams",
        "milligramme",
        "g",
        "gram",
        'grams'
        "gramme",
        "kg",
        "kilogram",
        "kilogramme",
        "x",
        "of",
        "mm",
        "millimetre",
        "millimeter",
        "cm",
        "centimeter",
        "centimetre",
        "m",
        "meter",
        "metre",
        "inch",
        "in",
        "milli",
        "centi",
        "deci",
        "hecto",
        "kilo",
        'teaspoon',
         'teaspoons',
         'tablespoon',
         'tablespoons',
         'cup',
         'cups',
         'ounce',
         'ounces',
         'fluid',
         'quart',
         'quarts',
         'gallon',
         'gallons',
         'pound',
         'pounds'
]

In [33]:
 words_to_remove = [
        "fresh",
        "minced",
        "chopped", 
        "a",
        "red",
        "bunch",
        "and",
        
        "or",
        "leaf",
        
        "large",
        "extra",
        "sprig",
        "ground",
        "handful",
        "free",
        "small",
        
        "virgin",
        "range",
        "from",
        "dried",
        "sustainable",
        "black",
        "peeled",
        "higher",
        "welfare",
        
        "for",
        "finely",
        "freshly",
        "sea",
        "quality",
        "white",
        "ripe",
        "few",
        "piece",
        "source",
        "to",
        "organic",
        "flat",
        "smoked",
        
        "sliced",
        "green",
        "picked",
        "the",
        "stick",
        "plain",
        "plus",
        "mixed",
        
        
        
        "your",
        
        "optional",
        
        "serve",
        
        "unsalted",
        "baby",
        
        "fat",
        "ask",
        "natural",
        "skin",
        "roughly",
        "into",
        "such",
        "cut",
        "good",
        "brown",
        "grated",
        "trimmed",
        
        "powder",
        "yellow",
        "dusting",
        "knob",
        "frozen",
        "on",
        "deseeded",
        "low",
        "runny",
        
        "cooked",
        "streaky",
        "nutmeg",
        "sage",
        "rasher",
        "zest",
        "pin",
        "groundnut",
        "breadcrumb",
        
        "halved",
        "grating",
        "stalk",
        "light",
        "tinned",
        "dry",
        "soft",
        "rocket",
        "bone",
        "colour",
        "washed",
        "skinless",
        "leftover",
        "splash",
        "removed",
        "dijon",
        "thick",
        "big",
        "hot",
        "drained",
        "sized",
        
        "watercress",
        "fishmonger",
        "english",
        
        
        "raw",
        
        "flake",
        
        
        "tbsp",
        "leg",
        "pine",
        "wild",
        "if",
        "fine",
        "herb",
        
        "shoulder",
        "cube",
        "dressing",
        "with",
        "chunk",
        "spice",
        "thumb",
        "garam",
        "new",
        "little",
        "punnet",
        
        "shelled",
        
        "other", "chopped",
        "salt",
        "olive",
        "taste",
        "can",
        "sauce",
        "water",
        "diced",
        "package",
        "italian",
        "shredded",
        "divided",
        
        "vinegar",
        "all",
        "purpose",
        "crushed",
        "juice",
        "more",
        "coriander",
        "bell",
        "needed",
        "thinly",
        "boneless",
        "half",
        
        "cubed",
        
        
        "jar",
        "seasoning",
        
        "extract",
        "sweet",
        "baking",
        "beaten",
        "heavy",
        "seeded",
        "tin",
        "vanilla",
        "uncooked",
        "crumb",
        "style",
        "thin",
        
        "coarsely",
        "spring",
        
        "cornstarch",
        "strip",
        
        "rinsed",
        "honey",
        
        "root",
        "quartered",
        "head",
        "softened",
        "container",
        "crumbled",
        "frying",
        "lean",
        "cooking",
        "roasted",
        "warm",
        "whipping",
        "thawed",
       
        "pitted",
        "sun",
        "kosher",
        "bite",
        "toasted",
        
        "split",
        "melted",
        "degree",
        "lengthwise",
        "romano",
        "packed",
        "pod",
       
        "rom",
        "prepared",
        "juiced",
        "fluid",
        "floret",
        "room",
        "active",
        "seasoned",
        "mix",
        "deveined",
        "lightly",
        
        "thai",
        "size",
        "unsweetened",
        "torn",
        "wedge",
        "sour",
        "basmati",
        
        "dark",
        "temperature",
        "garnish",
        
        "loaf",
        "shell",
        
        "canola",
        "parmigiano",
        "round",
        "canned",
        
        "crust",
        "long",
        "broken",
        "ketchup",
        "bulk",
        "cleaned",
        "condensed",
        "sherry",
        
        "cold",
        "soda",
        "cottage",
        "spray",
        "tamarind",
        "pecorino",
        "shortening",
        "part",
        "bottle",
        "sodium",
        "cocoa",
        "grain",
        "french",
        "roast",
        "stem",
        "link",
        "firm",
        "asafoetida",
        "mild",
        "dash",
        "boiling",
        
        "chopped",
        "skin off",
        "bone out" ,
        "from",
        "sustrainable",
        "sources",
    ]

In [34]:
sw = stopwords.words('english')
sw.extend(measures)
sw.extend(words_to_remove)

Time to create a function that will put these steps together to apply to a data frame series.

In [35]:
def ingredient_parser(recipe):
    
    #recipe = ' '.join(recipe)
    
    recipe = recipe.lower()
    
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenizer = RegexpTokenizer(pattern)
    recipe = tokenizer.tokenize(recipe)
    
    recipe_tagged = pos_tag(recipe)
    recipe_tagged
    
    new_list = []
    for i in recipe_tagged:
        if i[1]== 'NN':
            new_list.append(i)
        elif i[1]== 'NNS':
            new_list.append(i)
#         elif i[1] == 'JJ':
#             new_list.append(i)
    
    n_list = []
    for i in new_list:
        n_list.append(i[0])
     
    recipe = [token for token in n_list if token not in sw]
    recipe = ' '.join(recipe)
    return recipe

Testing the function.

In [36]:
sample = ingredient_parser(ingredients_list["Ingredients"][0])
sample

'potatoes egg whites reggiano pepper oil'

Sweet.

In [37]:
df['parsed'] = df['Ingredients'].apply(ingredient_parser)

In [38]:
df.head()

,Titles,URL,Ingredients,parsed
0,Potato Cheese Pancakes,https://cooking.nytimes.com/recipes/1000-potat...,1 1/2 pounds new potatoes (2 cups squeezed) 8 ...,potatoes egg whites reggiano pepper oil
1,Zucchini Pancakes,https://cooking.nytimes.com/recipes/1001-zucch...,1 1/2 pounds zucchini 8 ounces onion ( 1/2 cup...,egg whites thyme pepper
2,Fillets Of Flounder With Parsley And Mustard S...,https://cooking.nytimes.com/recipes/1002-fille...,Butter for greasing a pan Salt to taste if des...,butter pan pepper fillets wine leaves shallots...
3,Cucumbers And Carrots,https://cooking.nytimes.com/recipes/1003-cucum...,"2 large cucumbers, about 1 1/4 pounds 2 carrot...",cucumbers carrots shallots pepper cumin
4,Pasta With Fennel And Sardines,https://cooking.nytimes.com/recipes/1004-pasta...,2 packed cups mixed greens of fresh fennel fro...,greens fennel fronds fennel seeds fennel dillw...


Time to vectorize the parced data.

In [39]:
vec = CountVectorizer()
                    
X = vec.fit_transform(df['parsed'])

df_vec = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df_vec.head()

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ablespoon,absolut,abut,acacia,accent,accompaniments,achiote,acid,acidity,acorn,...,za,zabaglione,zebra,zester,zests,zinfandel,zinger,ziti,zucchini,zucchinis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# vocabulary = nltk.FreqDist()
# # This was done once I had already preprocessed the ingredients
# for ingredients in df['parced']:
#     ingredients = ingredients.split()
#     vocabulary.update(ingredients)
# for word, frequency in vocabulary.most_common(500):
#     print(f'{word};{frequency}')

In [41]:
# wordcloud = WordCloud(width = 800, height = 800,
#                 background_color ='white',
#                 stopwords = stopwords,
#                 min_font_size = 10).generate(words)
 
# # plot the WordCloud image                      
# plt.figure(figsize = (8, 8), facecolor = None)
# plt.imshow(wordcloud)
# plt.axis("off")
# plt.tight_layout(pad = 0)
 
# plt.show()

## Building the Recommendation System

First we need to set up an input system that will be able to be input into the models and get recommendations via some kind scoreing metric. To do this I need to create a one-row data frame with the same columns as the vectorized recipes. When the user inputs an ingredient list, the items on that list will get a 1 in the column if it exists in the dataframe. The new data frame will be turned into an array and that array can be compareed to all the arrays from the recipes to find the closest match. 

In [42]:
ingredients = input('Ingredient: ')

ingredients_list = ingredients.split()
df2 = pd.DataFrame([[0]*df_vec.shape[1]],columns=df_vec.columns)
df2.at[0,ingredients_list]=1
y = np.array(df2)
y.sum() # sanity check to make sure there are two 1s in the array

Ingredient: 


0

Cosine Similarity can be used to compare the arrays by measuring the cosine of the angle between the arrays.

In [43]:
# Utilize cosine_similarity from sklearn to return similarity scores based on cosine distance
cos_sim = cosine_similarity(df_vec, y,dense_output=True)
# Create a dataframe with similairty scores
cos_sim = pd.DataFrame(data=cos_sim, index=df_vec.index)
cos_sim

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
9016,0.0
9017,0.0
9018,0.0
9019,0.0


In [44]:
cos_sim.sort_values(by=[0],ascending=False)

,0
0,0.0
6017,0.0
6011,0.0
6012,0.0
6013,0.0
...,...
3001,0.0
3000,0.0
2999,0.0
2998,0.0


In [45]:
def recipe_recommend():
    
    ingredients = input('Ingredient: ')
    ingredients_list = ingredients.split()
    df2 = pd.DataFrame([[0]*df_vec.shape[1]],columns=df_vec.columns)
    df2.at[0,ingredients_list]=1
    n_recs = int(input('How many recommendations? '))
    
    y = np.array(df2)
    cos_sim = cosine_similarity(df_vec, y)
    cos_sim = pd.DataFrame(data=cos_sim, index=df_vec.index)
    cos_sim.sort_values(by = 0, ascending = False, inplace=True)
    results = cos_sim.index.values[1:n_recs+1]
    results_df = df.loc[results].drop('parsed',axis=1)
    results_df.reset_index(inplace=True)
    return results_df

In [46]:
pd.set_option('display.max_colwidth', None)

In [ ]:
recipe_recommend()

Ingredient: thyme


In [ ]:
# Instaniate and fit the model using merged dataframe
knn = NearestNeighbors(n_neighbors=5)
knn.fit(df_vec)
# Return results using .kneighbors attribute of knn model
n_recs = 5
results2 = knn.kneighbors(X=y, n_neighbors=n_recs+1, return_distance=False).flatten()
results2 = df.loc[results2].index.values[1:]
results2

In [ ]:
def recipe_recommend_knn():
    
    ingredients = input('Ingredient: ')
    ingredients_list = ingredients.split()
    df2 = pd.DataFrame([[0]*df_vec.shape[1]],columns=df_vec.columns)
    df2.at[0,ingredients_list]=1
    n_recs = int(input('How many recommendations? '))
    
    X = np.array(df2)
    results2 = knn.kneighbors(X, n_neighbors=5, return_distance=False).flatten()
    results2 = df_vec.iloc[results2].index.values[1:]
    results2
    results_df = df.loc[results2]
    results_df.reset_index(inplace=True)
    
    return results_df

In [ ]:
recipe_recommend_knn()

The knn recommendation system ended up finding blank entries in the data set. I will have to go back and remove the blank rows.

## Evaluation

The KNN methodology was not able to recommend recipes with the given ingredients so the cosine similaity methodology will be the scoreing metric behind the final system. When the KNN method was used it had returned some empty cells that I went back and removed from the data frame. Once that was done, the KNN method was able to return a non-empty set of recipes but the new cleaned data set affected the cosine similarity method to where the parsing of the recipes didn't work so the recommendations didn't include the input ingredients. So the null row were not removed in order to create appropriate recommendations.  

## Next Steps


The next steps for this project would be to refine the NLP methods to reduce the unnecessary words in the recipes. Some options to do this would be to use an already existing data set with ingredients and an API to use as a reference set to find names of common ingredients. Another way would be to implement a predictive model to classify words as a ingredient or not. Once the methods are refined I would like to build an app for this recommendation system.